# Dépendance

In [11]:
import seaborn as sns; sns.set()
import pandas as pd
import pandasql as psql
import numpy as np
from sklearn.model_selection import train_test_split as TTS
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.preprocessing import LabelEncoder as LE

# Variable

In [2]:
fraud = pd.read_csv('./data/Fraud_Data.csv')
ip = pd.read_csv('./data/IpAddress_to_Country.csv')

# Function

In [3]:
accuracy = lambda p,y: (p==y).sum()/p.shape[0]

In [4]:
def rfc_predict(X,Y,size=0.2, seed=777):
    splits = TTS(X,
                 Y,
                 train_size=size,
                 stratify=Y,
                 random_state=seed
                )
    X_train, X_test, Y_train, Y_test = splits
    rfc = RFC()
    rfc.fit(X_train, Y_train)
    prediction = rfc.predict(X_test)
    print(accuracy(prediction, Y_test))
    print()
    print(rfc.predict_proba(X_test))
    return True


# Notebook

In [5]:
fraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


In [6]:
ip.head()

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216.0,16777471,Australia
1,16777472.0,16777727,China
2,16777728.0,16778239,China
3,16778240.0,16779263,Australia
4,16779264.0,16781311,China


In [7]:
fraud = fraud.dropna()
ip = ip.dropna()

In [24]:
#for index, eachf in fraud.iterrows():
#    for _v, te in ip.iterrows():
#        if te.lower_bound_ip_address <= eachf.ip_address <= float(te.upper_bound_ip_address) : 
#            ip.loc[_v, 'ip_address'] = eachf.ip_address
#            print(str(_v) +" : "+ str(ip.iloc[_v]['ip_address']))

query = "SELECT * FROM fraud, ip WHERE fraud.ip_address BETWEEN ip.lower_bound_ip_address AND ip.upper_bound_ip_address"

In [26]:
data = psql.sqldf(query, locals())
data.head()

KeyboardInterrupt: 

In [28]:
# Label Encoder
le = LE()
clean = data.copy()
clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129146 entries, 0 to 129145
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   user_id                 129146 non-null  int64  
 1   signup_time             129146 non-null  object 
 2   purchase_time           129146 non-null  object 
 3   purchase_value          129146 non-null  int64  
 4   device_id               129146 non-null  object 
 5   source                  129146 non-null  object 
 6   browser                 129146 non-null  object 
 7   sex                     129146 non-null  object 
 8   age                     129146 non-null  int64  
 9   ip_address              129146 non-null  float64
 10  class                   129146 non-null  int64  
 11  lower_bound_ip_address  129146 non-null  float64
 12  upper_bound_ip_address  129146 non-null  int64  
 13  country                 129146 non-null  object 
dtypes: float64(2), int64

In [16]:
clean['device_id'] = le.fit_transform(clean['device_id'])
clean['browser'] = le.fit_transform(clean['browser'])
clean['sex'] = le.fit_transform(clean['sex'])
clean['source'] = le.fit_transform(clean['source'])
clean['country'] = le.fit_transform(clean['country'])
clean['purchase_time'] = pd.to_datetime(clean['purchase_time']).astype(int)/ 10**9
clean['signup_time'] = pd.to_datetime(clean['signup_time']).astype(int)/ 10**9

In [18]:
clean

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,lower_bound_ip_address,upper_bound_ip_address,country
0,22058,1.424819e+09,1.429325e+09,34,1286,2,0,1,39,7.327584e+08,0,7.298089e+08,734003199,40
1,333320,1.433710e+09,1.433728e+09,16,348,0,0,0,53,3.503114e+08,0,3.355443e+08,352321535,85
2,1359,1.420138e+09,1.420138e+09,15,1877,2,3,1,53,2.621474e+09,1,2.621440e+09,2621505535,85
3,221365,1.437463e+09,1.441824e+09,39,1014,0,4,1,45,4.155831e+08,0,4.154982e+08,415629311,85
4,159135,1.432188e+09,1.436429e+09,42,29,0,0,1,18,2.809315e+09,0,2.809266e+09,2809331711,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,293805,1.430795e+09,1.437781e+09,59,1437,0,1,1,35,1.554289e+09,0,1.551892e+09,1556086783,26
1996,43498,1.423401e+09,1.425492e+09,66,960,1,2,0,34,2.077439e+09,0,2.077229e+09,2077491199,3
1997,31827,1.431070e+09,1.438430e+09,47,539,2,4,1,31,2.809213e+09,0,2.809201e+09,2809266175,85
1998,261934,1.430187e+09,1.432861e+09,45,1351,1,2,0,30,1.533150e+09,0,1.533149e+09,1533280255,36


In [22]:
Y = clean['class']
X = clean.drop('class', axis='columns')

X.shape, Y.shape

((2000, 13), (2000,))

In [23]:
rfc_predict(X,Y)

0.946875

[[0.94 0.06]
 [0.99 0.01]
 [0.7  0.3 ]
 ...
 [0.99 0.01]
 [1.   0.  ]
 [0.98 0.02]]


True